<img src="images/DLI Header.png" alt="标题" style="width: 400px;"/>

自然语言处理 (NLP) 的关键用例——一些人可能甚至认为其是该领域的“圣杯”——是让机器自动将文本从一种语言翻译成另一种语言的能力。此领域被称为机器翻译 (MT)，当使用神经网络和深度学习时，它通常被称为神经网络机器翻译 (NMT)。

即使对于人类而言，语言翻译也往往颇具难度。不同语言之间存在多种差异，有些语言比其他语言更微妙。有一整个的研究领域——[语言类型学](https://en.wikipedia.org/wiki/Linguistic_typology)——致力于理解语言之间的差异。因此，翻译时需要充分了解源语言和目标语言以及二者之间的差异。

现今有大量文本数据可用，再加上 GPU 的计算能力，NMT 的使用量大幅增长是一件理所当然的事情。事实上，有多家知名公司面向这一领域提供各种产品。

在本次实验中，我们将着重探讨以下两点：

1. 为将人类可读日期格式转换成机器可读日期格式构建一套神经网络机器翻译系统
2. 利用注意力的概念改进模型

# 翻译迷失

想象一下，我们接到一个将一种语言（例如德语）翻译成英语的翻译任务。

下面是来自不同翻译模型的 5 种不同的翻译结果。虽然我们不知道原始句子，但我们可以看到，当下面的例子被视为一组时，可以很容易从中确定上下文。

译文 A：I ask him whether he will once again make a stand-up comedy tour.

译文 B：I ask him if he will again make a stand-up comedy tour.

译文 C：I wonder him if he will ever make a booth up comedy tour.

译文 D：I ask him if he will ever make a stand-up comedy tour ever.

译文 E：I ask him whether he will again make a stand-up comedy tour.

找出最差的译文应该相对比较容易，因为英文的直译意思不通顺。这显示，翻译时普遍都会面临困难。上下文十分重要。

In [ ]:
!python3 OpenSeq2Seq/run.py --config_file=OpenSeq2Seq/example_configs/nmt.json --logdir=/dli/data/noatt --mode=infer --inference_out=baseline.txt

让其运行，您可以打开 [baseline.txt](baseline.txt) 查看正在编写的译文。当看到“I wonder him if he will ever make a booth up com@@ ed@@ y@@ tour .”输出时，您可以按下 Stop（停止）或 Interrupt Kernel（中断内核）按钮。运行到这一行代码需要大约 3.5 分钟。本实验的最后部分将解释为什么译文由于使用了字节对编码 (BPE) 而包含了类似“com@@”这样的单词。您可以简单地删除这些字符来获得上面使用的句子

您可以看到这个模型生成的译文最差。但另一方面，确定最佳译文可能因人而异，因为这涉及到一些主观性。以译文 D 为例：在一句话中重复使用“ever”降低了其作为英文佳译的分数。事实证明，所有确认为最佳的结果都是借助注意力来实现的。这是一项关键认识，稍后我们将在讨论*注意力机制* 时回到该认识。

对于本次实验，我们将使用基于 Tensorflow 后端的 Keras 深度学习框架。运行以下单元来加载所有必需模块。

In [ ]:
from keras.layers import Embedding, Bidirectional
from keras.layers.core import *
from keras.layers.recurrent import LSTM
from keras.models import *
from keras.layers.merge import Multiply
from keras.utils import to_categorical
from keras.layers import TimeDistributed
from keras.backend import int_shape

import keras.backend as K
import numpy as np

from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date
from nmt_utils import *
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

## 将人类可读日期转换为机器可读日期

通常情况下，我们使用 NMT 将句子从源语言翻译成目标语言。在这部分内容中，我们将尝试一些稍微简单一点的东西，以便更深入地理解相关概念。
我们将建立一个模型，将人类可读日期，例如“the 29th of August 1958”、“03/30/1968”以及“24 JUNE 1987”翻译成标准的机器可读格式，例如“1958-08-29”、“1968-03-30”和“1987-06-24”。我们将使用美国日期编码格式 mm/dd/yyyy。查看 nmt_utils.py，看看所有日期格式。计算并确定这些格式的工作方式——稍后您将需要这方面的知识。

我们的数据集是一个包含 1 万个人类可读日期及等价的机器可读日期的语料库。运行以下单元来加载数据集并打印相关信息。

In [ ]:
m = 10000
dataset, human_vocab, machine_vocab, inv_machine_vocab = create_dataset(m)

Tx = 20
sources, targets = zip(*dataset)
sources = np.array([string_to_int(i, Tx, human_vocab) for i in sources])
targets = [string_to_int(t, Tx, machine_vocab) for t in targets]
targets = np.array(list(map(lambda x: to_categorical(x, num_classes=len(machine_vocab)), targets)))

以下数据块已加载：
- dataset：（人类可读日期，机器可读日期）成对列表
- human_vocab：所有在人类可读日期中使用的字符到其整数索引值的映射
- machine_vocab：与 `human_vocab` 类似，在机器可读日期中使用的所有字符到其整数索引值的映射
- inv_machine_vocab：`machine_vocab` 的翻转映射（即整数到字符）
- sources：已处理版本的 `dataset` 人类可读日期。每个字符都被来自 `human_vocab` 的对应索引所取代，结果序列已被填充为由 `Tx` 指定的长度。
- targets：已处理版本的 `dataset` 机器可读日期每个字符都被来自 `machine_vocab` 的对应索引所取代，序列已被填充为由 `Tx` 指定的长度。还请注意，为了用作模型中的标签，所有值均为独热编码。

以下单元从 `dataset` 中选择随机值，并显示日期的编码方式。多次运行，得到不同的值。

In [ ]:
index = np.random.randint(m)

print("sources shape          : ", sources.shape)
print("targets shape          : ", targets.shape)
print("Length of human_vocab  : ", len(human_vocab))
print("Length of machine_vocab: ", len(machine_vocab))

print("\n")
print("Human-readable date                 : ", dataset[index][0])
print("Machine-readable date               : ", dataset[index][1])
print("Pre-processed human-readable date   : ", sources[index])
print("Pre-processed machine-readable date : \n", targets[index])


## 模型架构

NMT 模型使用被称为 `encoder-decoder` 的网络架构。在更高层次，由该网络执行以下步骤：

1. 输入 - 在我们的例子中，一个人类可读日期被输入到网络的 `encoder` 部分。编码器输出的是一个或多个“状态”向量（也称为“上下文”向量）。这些状态向量编码输入数据的表征。如上所述，每个时间步均有它自己的上下文向量。或者，我们可以使用最终状态向量表征整个输入。
2. 编码器输出被输入到网络的“解码器”部分。解码器输出是最终输出，在我们的例子中是机器可读日期。解码器使用在状态向量中编码的数据创建输出。

由于我们模型的输入和输出均是序列，因此采用 RNN 同时作为编码器和解码器的基础是合理的。然后，我们可以端到端的方式训练编码器-解码器模型，以便两个网络同时学习相关特征。

### 编码器网络

<img src="images/enc.png" style="width:600;height:300px;"><br>
<caption><center>**图 1**：NMT 编码器网络</center></caption>

如图所示，编码器输入是人类可读日期文本。然后，序列中的每个字符都经过“嵌入层”处理。嵌入的目的是将字符的稀疏、独热表征转换为一个可以学习到关于该字符特征的密集表征。然后，该表征被输入至双向 LSTM (Bi-LSTM)。Bi-LSTM 的目的是从前到后以及从后到前查看特定序列。通过这种方式，网络为文本中的每个字符根据于其过去和未来创建一个上下文。最后，我们将 Bi-LSTM 的隐藏状态的序列作为我们的状态向量列表 `enc_out`。

### 解码器网络

<img src="images/dec.png" style="width:500;height:300px;"><br>
<caption><center>**图 2**：NMT 解码器网络</center></caption>

解码器接收 `enc_out` 作为其输入。这个序列本身通过 LSTM 运行，给出一组数字。我们使用 Softmax 函数来正则化这些数字，这样我们就可以将其作为概率。然后，我们可以选择最高概率的字符作为模型输出。

**练习**：执行下图中描述的 `model_simple_nmt()`。两个 LSTM 都使用 32 个单位，嵌入维度是 64。可能用到以下函数：[Input()](https://keras.io/layers/core/#input)、[Embedding()](https://keras.io/layers/embeddings/)、[LSTM()](https://keras.io/layers/recurrent/#lstm)、[Bidirectional()](https://keras.io/layers/wrappers/#bidirectional)、[Dense()](https://keras.io/layers/core/#dense)、[TimeDistributed()](https://keras.io/layers/wrappers/#timedistributed)、[Model()](https://keras.io/models/model/)。本手册的最后提供答案。

In [ ]:
def model_simple_nmt(human_vocab_size, machine_vocab_size, Tx = 20):
    """
    Simple Neural Machine Translation model
    
    Arguments:
    human_vocab_size -- size of the human vocabulary for dates, it will give us the size of the embedding layer
    machine_vocab_size -- size of the machine vocabulary for dates, it will give us the size of the output vector
    
    Returns:
    model -- model instance in Keras
    """
    
    ### START CODE HERE ###
    # Define the input of your model with a shape (Tx,)
    inputs = ##TODO## : Add Input layer code here
    
    # Define the embedding layer. Embedding dimension should be 64 and input_length set to Tx.
    input_embed = ##TODO## : Add Embeddings layer code here
    
    # Encode the embeddings using a bidirectional LSTM
    enc_out = ##TODO## : Add Bidirectional layer code here
    
    # Decode the encoding using an LSTM layer
    dec_out = ##TODO## : Add LSTM layer code here
    
    ### END CODE HERE ###
    
    # Apply Dense layer to every time step
    output = TimeDistributed(Dense(machine_vocab_size, activation='softmax'))(dec_out)
    
    # Create model instance 
    model = Model(input=[inputs], output=output)

    return model

运行以下单元来创建您的 `model` 并编译。

In [ ]:
# Create model
model = model_simple_nmt(len(human_vocab), len(machine_vocab), Tx)

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

运行以下单元来训练模型。

In [ ]:
model.fit([sources], targets, epochs=20, batch_size=128, validation_split=0.1, verbose=2)

## 结果

我们来尝试在一些简单的例子中运行我们的模型。

In [ ]:
EXAMPLES = ['3 May 1979', '5 Apr 09', '20th February 2016', 'Wed 10 Jul 2007', 'Saturday May 9 2018', 'March 3 2001', 'March 3rd 2001', '3rd of March 2001']

def run_examples(examples):
    for example in examples:
        source = string_to_int(example, Tx, human_vocab)
        prediction = model.predict(np.array([source]))
        prediction = np.argmax(prediction[0], axis = -1)
        output = int_to_string(prediction, inv_machine_vocab)
        print("source:", example)
        print("output:", ''.join(output))
    
run_examples(EXAMPLES)

请注意输入，以及模型为这些输入所犯的错误。我们可以猜测，字符串越长，出错越多。这和人类一样——当我们想要改善我们正在从事的特定任务的结果时，我们需要专注。因此，我们凭直觉想给神经网络提供同样的能力。这被称为“注意力”，我们将在下部分内容中进行讨论。

**练习**：现在轮到您创建您的测试集并给出类似的输出了。还记得我们要求您研究的那些数据格式吗？现在，请为以下测试集创建 5 种我们提供的“例子”中没有出现过的格式。然后填入代码，并在所有这些代码上运行预测，您会输出一个与我们所展示的字符串类似的输出字符串。

In [ ]:
##TODO## : Add your own formats and run the model on them here

# 使用注意力改善结果

想象一下，您正在将一本书中的一个段落从日语翻译成英语。您不可能阅读整段文字，然后合上书再翻译。您可能会先读一段文字。然后您会再读一遍，专注于每一部分，并且翻译。这就是注意力机制背后的想法。注意力是一种深度学习技术，用于帮助模型在需要时专注于输入的重要部分。现在，我们将对我们的数据 NMT 模型添加注意力。

## 注意力模型

<img src="images/NMT_Components.png" style="width:600;height:300px;"><br>
<caption><center>**图 3**：注意力 NMT 模型</center></caption>

正如我们所见，我们将人类可读日期输入到编码器的 Bi-LSTM 网络中，并将状态向量作为解码器输入。我们想要做的是——对于解码器的每一时间步，针对输出中的每个字符——让模型专注于输入中“目前”帮助最大的隐藏状态。在编码器和解码器之间添加注意力块就可以做到这一点。

<img src="images/Attention_mechanism.png" style="width:600;height:300px;"><br>
<caption><center>**图 4**：注意力执行</center></caption>

在上图中，我们可以看到注意力块的细节。每个隐藏状态在每个时间步中都有一个相关的权重。状态和权重要通过 $tanh$ 激活函数，使用 softmax 规范化。概率最大的隐藏状态将被用作解码器输入。这样一来，解码器在正确的时间接收到正确的输入。

我们来使用 softmax 和向量降维来实现一个简化的注意力模块。

**练习**：为下面单元中的注意力块填入代码。本手册的最后提供答案。

In [ ]:
def attention_3d_block(inputs):
    """
    Implement the attention block applied between two layers
    
    Argument:
    inputs -- output of the previous layer, set of hidden states
    
    Returns:
    output_attention_mul -- inputs weighted with attention probabilities
    """
    
    # Retrieve n_h and Tx from inputs' shape. Recall: inputs.shape = (m, Tx, n_h)
    Tx = int_shape(inputs)[1]
    n_h = int_shape(inputs)[2]
    
    ### START CODE HERE ###
    # Permute inputs' columns to compute "a" of shape (m, n_h, Tx)
    a = ##TODO## : Complete the Permute layer code here
    
    # Apply a Dense layer with softmax activation. It should contain Tx neurons. a.shape should still be (m, n_h, Tx).
    a = ##TODO## : Complete the Dense layer code here
    
    # Compute the mean of "a" over axis=1 (the "hidden" axis: n_h). a.shape should now be (m, Tx)
    a = ##TODO## : Complete the Lambda layer code here

    ### END CODE HERE ###
    
    # Repeat the vector "a" n_h times. "a" should now be of shape (m, n_h, Tx)
    a = RepeatVector(n_h)(a)
    
    # Permute the 2nd and the first column of a to get a probability vector of attention. a_probs.shape = (m, Tx, n_h)
    a_probs = Permute((2, 1), name='attention_vec')(a)
    
    # Apply the attention probabilities to the "inputs" by multiplying element-wise.
    output_attention_mul = Multiply(name='attention_mul')([inputs, a_probs])
    
    return output_attention_mul

##  评估注意力块的相关

为了确保您刚刚编写的激活块能够提供模型的关注点，我们运行一个小实验。

我们首先生成数据点 $(X, Y) = $ {$(x^{(i)}, y^{(i)})_{i=1...m}$}，其中：
- $y^{(i)}$ 是一个等于 0 或 1 的标签。
- $x^{(i)}$ 是形状为 $(T_x, n_h)$ 的矩阵，其中一列等于 $y^{(i)}$，剩下的列随机。这意味着对于某个时间步 t，x[t,:]= y。
- 进而，`X.shape =` $(m, T_x, n_h)$，`Y.shape =` $(m, 1)$。

我们来生成一些假数据，这些数据的其中一维是真答案。我们的注意力模型应该能学习到专注于这一列。

In [ ]:
def get_data_recurrent(n, time_steps, input_dim, attention_column=None):
    """
    Data generation. x is purely random except that it's first value equals the target y.
    In practice, the network should learn that the target = x[attention_column].
    Therefore, most of its attention should be focused on the value addressed by attention_column.
    :param n: the number of samples to retrieve.
    :param time_steps: the number of time steps of your series.
    :param input_dim: the number of dimensions of each element in the series.
    :param attention_column: the column linked to the target. Everything else is purely random.
    :return: x: model inputs, y: model targets
    """
    if attention_column is None:
        attention_column = np.random.randint(low=0, high=input_dim)
    x = np.random.standard_normal(size=(n, time_steps, input_dim))
    y = np.random.randint(low=0, high=2, size=(n, 1))
    x[:, attention_column, :] = np.tile(y[:], (1, input_dim))
    return x, y

运行下面的单元，看一些关于 X 和 Y 的例子。

In [ ]:
np.random.seed(1)
x, y = get_data_recurrent(n = 2, time_steps = 4, input_dim = 3, attention_column=None)
print("x.shape =", x.shape)
print("x =", x)
print()
print("y.shape =", y.shape)
print("y =", y)

数据集（X，Y）可用于观察注意力的影响。实际上，我们将尝试一个假定 x 将预测 y 的二元分类器。由于注意力机制，网络应该知道，只有 x 的一个时间步可用于预测 y，其余随机。模型应该只专注于该特定时间步。运行下面的单元以加载数据集，注意力放在第三个时间步上。

In [ ]:
X, Y = get_data_recurrent(n = 10000, time_steps = 20, input_dim = 2, attention_column = 3)

###  为 LSTM 添加注意力

注意力块可以放置在网络中的不同位置。最常见的位置是在 LSTM 之前和之后。下面我们提供两种其注意力块应用于 LSTM 之前和之后的不同模型。

In [ ]:
def model_attention_applied_before_lstm(Tx, n_h):
    """
    Model with attention applied BEFORE the LSTM
        
    Returns:
    model -- Keras model instance
    """
    
    # Define the input of your model with a shape (Tx,)
    inputs = Input(shape=(Tx, n_h,))
    # Add the attention block
    attention_mul = attention_3d_block(inputs)
    # Pass the inputs in a LSTM layer, return the sequence of hidden states
    attention_mul = LSTM(32, return_sequences=False)(attention_mul)
    # Apply Dense layer with sigmoid activation the output should be a single number.
    output = Dense(1, activation='sigmoid')(attention_mul)
    # Create model instance 
    model = Model(input=[inputs], output=output)
    
    return model

In [ ]:
def model_attention_applied_after_lstm(Tx, n_x):
    """
    Model with attention applied AFTER the LSTM
    
    Returns:
    model -- Keras model instance
    """
    
    # Define the input of your model with a shape (Tx,)
    inputs = Input(shape=(Tx, n_x,))
    # Pass the inputs in a LSTM layer, return the sequence of hidden states
    lstm_out = LSTM(32, return_sequences=True)(inputs)
    # Add the attention block
    attention_mul = attention_3d_block(lstm_out)
    # Flatten the output of the attention block
    attention_mul = Flatten()(attention_mul)
    # Apply Dense layer with sigmoid activation the output should be a single number.
    output = Dense(1, activation='sigmoid')(attention_mul)
    # Create model instance 
    model = Model(input=[inputs], output=output)
    
    return model

我们将首先对其注意力应用于 LSTM 之后的模型进行训练。之后，您可以回到这里并尝试训练其注意力应用于 LSTM 之前的模型。将注意力直接应用于输入的 LSTM 之前的一个优点是更容易推理。概率分布仅跨越输入维度。然而，注意力更常应用于 LSTM 输出中，使得维度空间更复杂，以致难以解释和推理。

In [ ]:
m = model_attention_applied_after_lstm(Tx = 20, n_x = 2)

运行下面的单元来编译模型。

In [ ]:
m.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

运行下面的单元来训练模型。

In [ ]:
m.fit([X], Y, epochs=10, batch_size=16, validation_split=0.1, verbose=2)

我们现在定义 `get_activations` 函数，然后看一下我们的层的激活。这有助于我们确定注意力如何影响结果。

In [ ]:
def get_activations(model, inputs, layer_name=None):
    """
    For a given Model and inputs, find all the activations in specified layer
    If no layer then use all layers
    
    Returns:
    activations from all the layer(s)
    """
    activations = []
    inp = model.input
    if layer_name is None:
        outputs = [layer.output for layer in model.layers]
    else:
        outputs = [layer.output for layer in model.layers if layer.name == layer_name]  # all layer outputs
    funcs = [K.function([inp] + [K.learning_phase()], [out]) for out in outputs]  # evaluation functions
    layer_outputs = [func([inputs, 1.])[0] for func in funcs]
    for layer_activations in layer_outputs:
        activations.append(layer_activations)
    return activations

定义以下所有变量和参数：

In [ ]:
# Set the input dimensions
INPUT_DIM = 2

# Set time steps
TIME_STEPS = 20

# if True, the attention vector is shared across the input_dimensions where the attention is applied.
# Set whether the attention vector is shared
SINGLE_ATTENTION_VECTOR = False

# Set the attention model in relation to LSTM
APPLY_ATTENTION_BEFORE_LSTM = False

# Set the size of the dataset
N = 300000

运行以下单元来编译模型。

In [ ]:
inputs_1, outputs = get_data_recurrent(N, TIME_STEPS, INPUT_DIM, attention_column=3)

if APPLY_ATTENTION_BEFORE_LSTM:
    m = model_attention_applied_after_lstm(Tx = 20, n_x = 2)
else:
    m = model_attention_applied_before_lstm(Tx = 20, n_h = 2)

m.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print(m.summary())

我们使用 `attention_column=3` 生成 30 万个训练示例，并调用 `get_activations`。

In [ ]:
m.fit([inputs_1], outputs, epochs=1, batch_size=512, validation_split=0.1, verbose=2)

attention_vectors = []
for i in range(300):
    # Generate one training example (x, y), the attention column can be on any time-step.
    testing_inputs_1, testing_outputs = get_data_recurrent(1, TIME_STEPS, INPUT_DIM, attention_column=3)
    # Extract the attention vector predicted by the model "m" on the training example "x".
    attention_vector = np.mean(get_activations(m,
                                               testing_inputs_1,
                                               layer_name='attention_vec')[0], axis=2).squeeze()
    # Append the attention vector to the list of attention vectors
    assert (np.sum(attention_vector) - 1.0) < 1e-5
    attention_vectors.append(attention_vector)
# Compute the average attention on every time-step

attention_vector_final = np.mean(np.array(attention_vectors), axis=0)

通过运行下面的单元，生成图，以显示注意力的应用位置及其对模型结果的影响。

In [ ]:
# plot part.
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

pd.DataFrame(attention_vector_final, columns=['attention (%)']).plot(kind='bar',
                                                                     title='Attention Mechanism as '
                                                                           'a function of input'
                                                                           ' dimensions.')
plt.show()

**练习**：使用在 LSTM 之前的注意力块重复所有步骤。请注意执行此操作需要改动的所有变量和代码。

## 为 NTM 添加注意力

对于上面的小实验，显然不需要注意力。然而，如果没有注意力，我们将无法在日期翻译任务上取得进展。

现在，我们将为我们之前看到的日期 NMT 模型添加注意力。

**练习**：重新实现 `model_simple_nmt()`，这次添加注意力。看看您是否可以通过添加单位、层等因素得到更好的结果。

In [ ]:
def model_attention_nmt(human_vocab_size, machine_vocab_size, Tx = 20):
    """
    Attention Neural Machine Translation model
    
    Arguments:
    human_vocab_size -- size of the human vocabulary for dates, it will give us the size of the embedding layer
    machine_vocab_size -- size of the machine vocabulary for dates, it will give us the size of the output vector
    
    Returns:
    model -- model instance in Keras
    """

    # Define the input of your model with a shape (Tx,)
    inputs = Input(shape=(Tx,))
    
    # Define the embedding layer. This layer should be trainable and the input_length should be set to Tx.
    input_embed = Embedding(human_vocab_size, 2*32, input_length = Tx, trainable=True)(inputs)

    ### START CODE HERE ###    
    # Encode the embeddings using a bidirectional LSTM
    enc_out = ##TODO## : Add Bidirectional layer code here
    
    # Add attention
    attention = ##TODO## : Add attention block
    
    # Decode the encoding using an LSTM layer
    dec_out = ##TODO## : Add LSTM layer code here
 
    ### END CODE HERE ###
    
    # Apply Dense layer to every time steps
    output = TimeDistributed(Dense(machine_vocab_size, activation='softmax'))(dec_out)
    
    # Create model instance 
    model = Model(input=[inputs], output=output)
    
    return model

<img src="images/enc_attention.png" style="width:600;height:300px;"><br>
<caption><center>**图 5**：注意力模型</center></caption>

In [ ]:
# Create model
model_att = model_attention_nmt(len(human_vocab), len(machine_vocab))

# Compile model
model_att.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

<img src="images/train_infer.png" style="width:600;height:300px;"><br>
<caption><center>**图 6**：训练和推理</center></caption>

通过运行以下单元来训练模型。

In [ ]:
model_att.fit([sources], targets, epochs=100, batch_size=128, validation_split=0.1, verbose=2)

### 结果

In [ ]:
example = "Nov 11 1998"
source = string_to_int(example, 20, human_vocab)
prediction = model_att.predict(np.array([source]))
prediction = np.argmax(prediction[0], axis = -1)
output = int_to_string(prediction, inv_machine_vocab)
print("source:", example)
print("output:", ''.join(output))

### 可视化注意力

如果我们在本试验中运行初始模型，我们的注意力映射图很可能类似于以下：
<img src="images/poorly_trained_model.png" style="width:600;height:300px;"><br>
<caption><center>**图 7**：简单的注意力地图</center></caption>

“3 May 1979”的译文基本上没有意义，除了 0 和 1 外，什么也没有，没有有效的日期字符串。注意力只被应用于单个字符 y，因此翻译无法进行。

然而，对于一个有效训练的模型来说，注意力可能类似于以下映射：
<img src="images/date_attention.png" style="width:600;height:300px;"><br>
<caption><center>**图 8**：完整的注意力地图</center></caption>

这个模型可以处理难度更大的翻译任务，例如“Saturday 9 May 2018”。请注意注意力如何帮助模型完全忽略对我们的任务没有帮助的“Saturday”。“9”已经被翻译成“09”，通过“将更多注意力放在”“M”字符上，“May”已经被正确翻译成“05”。年的翻译主要要求模型专注于“18”，以返回“2018”的结果。

下面的 `attention_map` 函数将为我们的模型生成类似的显示：

In [ ]:
def attention_map(model, input_vocabulary, inv_output_vocabulary, text):
    """
        visualization of attention map
    """
    # encode the string
    encoded = string_to_int(text, 20, input_vocabulary)

    # get the output sequence
    prediction = model.predict(np.array([encoded]))
    predicted_text = np.argmax(prediction[0], axis=-1)
    predicted_text = int_to_string(predicted_text, inv_output_vocabulary)

    text_ = list(text)
    # get the lengths of the string
    input_length = len(text)
    output_length = predicted_text.index('<pad>') if '<pad>' in predicted_text else len(predicted_text)
    # get the activation map
    attention_vector = get_activations(model, [encoded], layer_name='attention_vec')[0].squeeze()
    activation_map = attention_vector[0:output_length, 0:input_length]
    
    plt.clf()
    f = plt.figure(figsize=(8, 8.5))
    ax = f.add_subplot(1, 1, 1)

    # add image
    i = ax.imshow(activation_map, interpolation='nearest', cmap='gray')

    # add colorbar
    cbaxes = f.add_axes([0.2, 0, 0.6, 0.03])
    cbar = f.colorbar(i, cax=cbaxes, orientation='horizontal')
    cbar.ax.set_xlabel('Probability', labelpad=2)

    # add labels
    ax.set_yticks(range(output_length))
    ax.set_yticklabels(predicted_text[:output_length])

    ax.set_xticks(range(input_length))
    ax.set_xticklabels(text_[:input_length], rotation=45)

    ax.set_xlabel('Input Sequence')
    ax.set_ylabel('Output Sequence')

    # add grid and legend
    ax.grid()

    f.show()

运行 `attention_map` 以生成图形：

In [ ]:
attention_map(model_att, human_vocab, inv_machine_vocab, example)

# 真实用例

**在执行这部分之前先运行 Kernel->Restart，这样 GPU 的内存就会完全释放出来。**


我们现在可以更好地理解注意力的用途了。接下来，我们将尝试运行一个真实的神经网络机器翻译模型，将德语翻译成英语。

但首先，我们需要讨论机器翻译以及其他 NLP 任务中的一个重点：BPE 表征。

### BPE
BPE 表示字节对编码。它意味着普通字节对——在我们的例子中是指双字母字符——被一个永远也不会在语料库中出现的字节所取代。例如，在我们的语料库中，我们从来没有见过“#”字符，因此我们可以用它来表示“ie”等一些典型的双字母。然而，实际上，所有可打印字符都已被使用，因此我们为 BPE 使用代码页面的不可打印部分。为了实际打印文本，我们需要将其重新格式化。所以您会在文本中看到“@@”，它们是重新规范化时遗留的字符。

这里有一个德语示例文本，将使用我们的模型翻译成英语。

In [ ]:
!head /dli/data/wmt/newstest2015.tok.bpe.32000.de

实际使用的架构是：
![](../2017-09-14_23-11-48.png)
它比我们之前的日期任务稍微复杂一点。我们又有了一个编码器-解码器架构，但现在注意力被应用于整个输入而不仅仅是其中一部分。


我们来看看结果如何。

In [ ]:
!python3 OpenSeq2Seq/run.py --config_file=OpenSeq2Seq/example_configs/nmt.json --logdir=/dli/data/nmt --mode=infer --inference_out=pred.txt

打开 [pred.txt](pred.txt)，对比 [baseline.txt](baseline.txt)，看看注意力对整体翻译质量造成的影响。跑过更多epoch 之后，效果将会提高，您可能会得到本试验第一部分中您认为最佳的译文。

## 评估 - BLEU 分数

在这最后一部分中，您将利用双语评估替补 (BLEU) 分数来评估翻译的效果。完全不匹配的结果得 0.0 分，完美的翻译得 1.0 分。有各种公式可应用于句子、语料库或任何长度的翻译，但对比这些公式是一种好的做法。我们的日期翻译示例很简单，但 BLEU 分数也可以用来概括一个人如何评价语言翻译的翻译结果。

<img src="images/BLEU.png" style="width:600;height:300px;">

BLEU 可被描述为单个形符和形符序列（2、3 或 4）的重叠——即 $precision_i$。
此外，该公式也用到了过短惩罚 $\frac{output-length}{reference - length}$。

我们把一个真值分解成一次一个字符，预测值同理。BLEU 分数使用 1-4 个 ngram，因此我们需要在预测中至少提供 4 个 ngram。对于字符串 2007-07-11，我们可以将其分解成形符 2007、07 和 11。对于精确匹配，分数是 1.0，但对于丢掉了一个字符这样的小的差异，分数会低于其应得值。我们会调用实现在 `str_score` 这个函数中的评分方法。

或者，我们可以通过标记每个字符，包括破折号，将其转换成 ['2', '0', '0', '7', '-', '0', '7', '-', '1', '1'] 列表，以获得更高的分数。这应在 `char_score` 中实现。我们比较一下两种实施结果，以及其计算得出的 BLEU 分数。

供您参考： sentence_bleu 函数的参数如下：	
references (list(list(str))) - 参考翻译列表<br>
hypothesis (list(str)) - 假设翻译

### 生成正确的 BLEU 分数

修改 [my_bleu.py](../../../../edit/tasks/task3/task/my_bleu.py) 中的计算两种类型的 BLEU 分数的准备和比较字符串的代码。保存更改，以修复 BLEU 分数计算。

提示：查看实际输出，并和预期的不同类型的BLEU分数进行对比。

**完成此任务后，请返回您用来打开这个笔记本的浏览器页面，然后单击“ASSESS TASK”（评估任务）按钮。如果您已经正确实施了 BLEU 分数，并通过了所有其他评估问题，您将获得 NLP 深度学习能力的认证。**

In [ ]:
# Test your implementation
# Kernel->restart will ensure the latest version of my_bleu.py is imported if you made any changes to the code to re-test

import my_bleu as mb

output1 = '2007-07-11<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'
expected = '2007-07-11'

print(mb.str_score(output1, expected))    #expecting 1.0
print(mb.char_score(output1, expected)) # expecting 1.0

output2 = '2007-07-12<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>' #it's 12 instead of 011

print(mb.str_score(output2, expected))    #expecting 0.7598356856515925
print(mb.char_score(output2, expected)) #expecting 0.8801117367933934

# [可选]束搜索（Beam Search）实验

正如我们所见，注意力机制改变了在给定序列中关注哪个字符的概率。我们还可以应用其他策略来改善翻译结果。束搜索是一种很常见的尝试。它的工作方式是为给定输入返回最有可能的输出“序列”列表。它首先通过探索所有可能的下一个步骤，然后从所有可选项中选出概率最大的 $k$ 个选项。这些候选项形成方向“束”，算法通过每个候选项的概率序列来扩大搜索范围。

束越大，模型性能越好。由于有更多候选序列，这意味着找到正确答案的可能性增加。然而，它需要更多的解码，这意味着需要对翻译结果的质量和准确性进行直接权衡。使用序列中概率最大的下一个词进行束搜索可以产生好的结果。束搜索将生成结果序列，从第一个词开始，然后追加，并在每一步探索所有候选项（束尺寸）。

我们已经开始了束搜索，以使用以下代码生成样本：

In [ ]:
m = model_attention_nmt(len(human_vocab), len(machine_vocab))

m.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(m.summary())

In [ ]:
inputs, targets = zip(*dataset)
inputs = np.array([string_to_int(i, 20, human_vocab) for i in inputs])
targets = [string_to_int(t, 20, machine_vocab) for t in targets]
targets = np.array(list(map(lambda x: to_categorical(x, num_classes=len(machine_vocab)), targets)))

In [ ]:
m.fit([inputs], targets, epochs=1, batch_size=64, validation_split=0.1)

In [ ]:
def keras_rnn_predict(samples, empty=human_vocab["<pad>"], rnn_model=m, maxlen=30):
    """for every sample, calculate probability for every possible label
    you need to supply your RNN model and maxlen - the length of sequences it can handle
    """
    data = sequence.pad_sequences(samples, maxlen=maxlen, value=empty)
    return rnn_model.predict(data, verbose=0)

在这里，我们将尝试从我们的模型中生成一些日期，以说明束搜索的特性：

In [ ]:
def beamsearch(predict=keras_rnn_predict, k=1, maxsample=10, 
               use_unk=False, 
               oov=human_vocab["<unk>"], 
               empty=human_vocab["<pad>"], 
               eos=human_vocab["<unk>"]):
    """return k samples (beams) and their NLL scores, each sample is a sequence of labels,
    all samples starts with an `empty` label and end with `eos` or truncated to length of `maxsample`.
    You need to supply `predict` which returns the label probability of each sample.
    `use_unk` allow usage of `oov` (out-of-vocabulary) label in samples
    """
    
    dead_k = 0 # samples that reached eos
    dead_samples = []
    dead_scores = []
    live_k = 1 # samples that did not yet reached eos
    live_samples = [[empty]]
    live_scores = [0]

    while live_k and dead_k < k:
        # for every possible live sample calc prob for every possible label 
        probs = predict(live_samples, empty=empty)

        # total score for every sample is sum of -log of word prb
        cand_scores = np.array(live_scores)[:,None] - np.log(probs)
        if not use_unk and oov is not None:
            cand_scores[:,oov] = 1e20
        cand_flat = cand_scores.flatten()

        # find the best (lowest) scores we have from all possible samples and new words
        ranks_flat = cand_flat.argsort()[:(k-dead_k)]
        live_scores = cand_flat[ranks_flat]

        # append the new words to their appropriate live sample
        voc_size = probs.shape[1]
        live_samples = [live_samples[r//voc_size]+[r%voc_size] for r in ranks_flat]

        # live samples that should be dead are...
        zombie = [s[-1] == eos or len(s) >= maxsample for s in live_samples]
        
        # add zombies to the dead
        dead_samples += [s for s,z in zip(live_samples,zombie) if z]  # remove first label == empty
        dead_scores += [s for s,z in zip(live_scores,zombie) if z]
        dead_k = len(dead_samples)
        # remove zombies from the living 
        live_samples = [s for s,z in zip(live_samples,zombie) if not z]
        live_scores = [s for s,z in zip(live_scores,zombie) if not z]
        live_k = len(live_samples)

    return live_samples + dead_samples, live_scores + dead_scores

# 总结

在本实验中，我们讨论了如何使用深度学习来执行机器学习。我们实验了一个简单的日期转换问题：将日期从人类可读转换成机器可读形式。然后，我们使用注意力改进了模型。最后，我们探索了束搜索的使用。

在本实验中，我们专注于“监督式”机器翻译。这直接表明我们需要标记好的语料库。然而，对于很少或没有标记数据的语言来说，这不太可能实现。一个令人着迷的研究途径是使用“无人监督”的 MT，它不需要这样的语料库。

机器翻译是一种使能工具，使世界各地的人们和文化更紧密地联系在一起。您在这个实验里看到的工具和技术可以帮助您实现这一目标。

__鸣谢__：基于 Philippe Remy 的 keras-visualize-activations 代码

URL: https://github.com/philipperemy/keras-visualize-activations

日期翻译的概念借鉴于 https://github.com/datalogue/keras-attention.

我们的示例中使用了https://github.com/NVIDIA/OpenSeq2Seq ， Seq2Seq 模型 NVIDIA 的实现。


### [代码练习答案](task3_Answers.ipynb)

<img src="images/DLI Header.png" alt="Header" style="width: 400px;"/>